In [18]:
import os
os.chdir('/Users/furuku/Documents/thesis/2020 Thesis - O_D Data ')
os.getcwd()

'/Users/furuku/Documents/thesis/2020 Thesis - O_D Data '

In [19]:
import pandas as pd

import json

In [20]:
from elasticsearch import Elasticsearch 
from elasticsearch.helpers import streaming_bulk
from elasticsearch.helpers import scan

es=Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [21]:
es.ping()

True

In [22]:
def create_index(index: str):
    es.indices.create(
        index=index,
        body={
            "settings": {"number_of_shards": 1},
            "mappings": {
                "properties": {
                    "Travel Month": {"type": "date"},
                    "Orig": {"type": "keyword"},
                    "Stop1": {"type": "keyword"},
                    "Stop2": {"type": "keyword"},
                    "Stop3": {"type": "keyword"},
                    "Dest": {"type": "keyword"},
                    "passenger": {"type": "integer"},
                    "Region_orig": {"type": "keyword"},
                    "country_orig": {"type": "keyword"},
                    "Region_dest": {"type": "keyword"},
                    "country_dest": {"type": "keyword"},

                    
                }
            },
        },
        ignore=400,
    )


In [23]:
def gendata(table,inp):
    
    for row in table:
        
        yield {
            
            "_index" : inp,
#             '_type' : '_doc',
            '_source' : row
            
        }

In [24]:
def preprocess_connect(data):
    
    data = data.rename(columns={'Stop #1':'Stop1','Stop #2':'Stop2','Stop #3':'Stop3','Stop #4':'Stop4','Stop #5':'Stop5','Reported + Est. Pax': 'passenger'})
    data = data.drop(['Stop4','Stop5','Al 5','Al 6','Op Al 5','Op Al 6'],axis=1)
    data = data[(data['Travel Month'].map(type)  != str) &  (data['Travel Month'].notnull()) ]
    data = data.dropna(subset=['Orig','Dest'])
    data = data[['Travel Month','Orig','Stop1','Stop2','Stop3', 'Dest','passenger']]
    data['passenger'] = data['passenger'].astype(int)
    
    if (data['Travel Month'].dtypes != ('O')):
        data['Travel Month'] = data['Travel Month'].dt.strftime('%Y-%m-%d')
    else:
        data['Travel Month'] = pd.to_datetime(data['Travel Month'], format='%Y-%m-%d')
        data['Travel Month'] = data['Travel Month'].dt.strftime('%Y-%m-%d')
    
    
    return data

In [25]:
# def sum_passenger(sin_direct):
#     sin_direct['passenger'] = sin_direct.groupby(['Orig', 'Stop1','Stop2','Stop3','Dest'], dropna=False)['passenger'].transform('sum')
#     sin_direct['count'] = sin_direct.groupby(['Orig', 'Stop1','Stop2','Stop3','Dest'], dropna=False)['passenger'].transform('count')
#     sin_direct = sin_direct.drop_duplicates()
#     sin_direct = sin_direct.sort_values('passenger', ascending=False)
    
#     return sin_direct

In [26]:
def join_region(data,region):
    data = data.join(region, on=['Orig'])
    data = data.rename(columns={"Global Region": "Region_orig", "Country Name": "country_orig"})
    data = data.join(region, on=['Dest'])
    data = data.rename(columns={"Global Region": "Region_dest", "Country Name": "country_dest"})
    
    return data
    

In [3]:
y2005 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2005 BKK as O_D.xlsx") 
y2006 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2006 BKK as O_D.xlsx") 
y2007 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2007 BKK as O_D.xlsx") 
y2008 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2008 BKK as O_D.xlsx") 
y2009 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2009 BKK as O_D.xlsx") 
y2010 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2010 BKK as O_D.xlsx") 
y2011 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2011 BKK as O_D.xlsx") 
y2012 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2012 BKK as O_D.xlsx") 
y2013 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2013 BKK as O_D.xlsx") 
y2014 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2014 BKK as O_D.xlsx") 
y2015 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2015 BKK as O_D.xlsx") 
y2016 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2016 BKK as O_D.xlsx") 
y2017 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_2017 BKK as O_D.xlsx") 
y2018_19 = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_Jan2018-Dec2019 BKK as O_D.xlsx") 

In [4]:
y2005_07_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2005-2007 BKK as STOP.xlsx")  
y2008_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2008 BKK as STOP.xlsx") 
y2009_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2009 BKK as STOP.xlsx") 
y2010_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2010 BKK as STOP.xlsx") 
y2011_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2011 BKK as STOP.xlsx") 
y2012_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2012 BKK as STOP.xlsx") 
y2013_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2013 BKK as STOP.xlsx") 
y2014_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2014 BKK as STOP.xlsx") 
y2015_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2015 BKK as STOP.xlsx") 
y2016_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2016 BKK as STOP.xlsx") 
y2017_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_2017 BKK as STOP.xlsx") 
y2018_19_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_Jan2018-Dec2019 BKK as STOP.xlsx") 

In [5]:
# data = pd.read_excel("./BKK As O_D/Traffic_Per_O_D_Report_Jan2018-Dec2019 BKK as O_D.xlsx")
# data_stop = pd.read_excel("./BKK As STOP or Connecti Point/Traffic_Per_O_D_Report_Jan2018-Dec2019 BKK as STOP.xlsx")
country = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/1.-Airport_Lookup_Country-Region.xlsx")
thai = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/Thailand_Airport_Lookup.xlsx")
# y2019 = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/2019_Schedule_Daily_Extract_Report_8030.xlsx")

In [27]:
y2018 = y2018_19[y2018_19['Travel Month'].dt.year == 2018]
y2019 = y2018_19[y2018_19['Travel Month'].dt.year == 2019]
y2005_stop = y2005_07_stop[y2005_07_stop['Travel Month'].dt.year == 2005]
y2006_stop = y2005_07_stop[y2005_07_stop['Travel Month'].dt.year == 2006]
y2007_stop = y2005_07_stop[y2005_07_stop['Travel Month'].dt.year == 2007]
y2018_stop = y2018_19_stop[y2018_19_stop['Travel Month'].dt.year == 2018]
y2019_stop = y2018_19_stop[y2018_19_stop['Travel Month'].dt.year == 2019]


In [28]:
data_airport = country.append(thai)
region = data_airport[['Code', 'Global Region','Country Name']]
region = region.drop_duplicates()
region = region.set_index('Code')
region = region.dropna()

In [30]:
test = preprocess_connect(y2019)
test = join_region(test,region)
test

,Travel Month,Orig,Stop1,Stop2,Stop3,Dest,passenger,Region_orig,country_orig,Region_dest,country_dest
226323,2019-01-01,SIN,NaN,NaN,NaN,BKK,12635,Asia,Singapore,Asia,Thailand
226324,2019-01-01,BKK,NaN,NaN,NaN,SIN,11260,Asia,Thailand,Asia,Singapore
226325,2019-01-01,HAK,SIN,NaN,NaN,BKK,20,Asia,China,Asia,Thailand
226326,2019-01-01,CGK,SIN,NaN,NaN,BKK,161,Asia,Indonesia,Asia,Thailand
226327,2019-01-01,PLM,SIN,NaN,NaN,BKK,38,Asia,Indonesia,Asia,Thailand
...,...,...,...,...,...,...,...,...,...,...,...
456645,2019-12-01,JED,RUH,DEL,NaN,BKK,1,Middle East,Saudi Arabia,Asia,Thailand
456646,2019-12-01,AHB,RUH,DEL,NaN,BKK,1,Middle East,Saudi Arabia,Asia,Thailand
456647,2019-12-01,GIZ,RUH,DEL,NaN,BKK,1,Middle East,Saudi Arabia,Asia,Thailand
456648,2019-12-01,MED,RUH,DEL,NaN,BKK,1,Middle East,Saudi Arabia,Asia,Thailand


In [61]:
tt = pd.DataFrame(test.groupby('country_orig')['passenger'].apply(sum)).reset_index()
tt = tt.rename(columns={'country_orig': 'name','passenger': 'value'})
tt.to_dict('records')

[{'name': 'Afghanistan', 'value': 1946},
 {'name': 'Albania', 'value': 644},
 {'name': 'Algeria', 'value': 2673},
 {'name': 'Angola', 'value': 958},
 {'name': 'Antigua and Barbuda', 'value': 16},
 {'name': 'Argentina', 'value': 8999},
 {'name': 'Armenia', 'value': 3237},
 {'name': 'Aruba', 'value': 34},
 {'name': 'Australia', 'value': 346568},
 {'name': 'Austria', 'value': 111018},
 {'name': 'Azerbaijan', 'value': 2391},
 {'name': 'Bahamas', 'value': 177},
 {'name': 'Bahrain', 'value': 44944},
 {'name': 'Bangladesh', 'value': 158559},
 {'name': 'Barbados', 'value': 19},
 {'name': 'Belarus', 'value': 3166},
 {'name': 'Belgium', 'value': 85273},
 {'name': 'Belize', 'value': 20},
 {'name': 'Benin', 'value': 290},
 {'name': 'Bermuda', 'value': 97},
 {'name': 'Bhutan', 'value': 44000},
 {'name': 'Bolivia', 'value': 87},
 {'name': 'Bosnia and Herzegovina', 'value': 1552},
 {'name': 'Botswana', 'value': 445},
 {'name': 'Brazil', 'value': 21485},
 {'name': 'Brunei Darussalam', 'value': 25860},

In [31]:
index = ['y2005','y2006','y2007','y2008','y2009','y2010','y2011',
         'y2012','y2013','y2014','y2015','y2016','y2017','y2018',
         'y2019','y2005_stop','y2006_stop','y2007_stop','y2008_stop','y2009_stop','y2010_stop','y2011_stop',
         'y2012_stop','y2013_stop','y2014_stop','y2015_stop','y2016_stop','y2017_stop','y2018_stop','y2019_stop']

table = [y2005, y2006, y2007, y2008, y2009, y2010, y2011,
         y2012, y2013, y2014, y2015, y2016, y2017, y2018,
         y2019, y2005_stop, y2006_stop, y2007_stop, y2008_stop, y2009_stop, y2010_stop, y2011_stop,
         y2012_stop, y2013_stop, y2014_stop, y2015_stop, y2016_stop, y2017_stop, y2018_stop, y2019_stop]

for idx,name in enumerate(table):
    
    print("Process in " + index[idx] )
    
    tmp = name.copy()
    add_elastic = preprocess_connect(tmp)
    add_elastic = join_region(add_elastic,region)
    
    json_str = add_elastic.to_json(orient='records')
    json_records = json.loads(json_str) 
    
    inserted = 0


    for success, info in streaming_bulk(
        client=es, actions=gendata(json_records,index[idx]),request_timeout=50
        ):
        inserted += success
        
#     print(inserted)
    
    if add_elastic.shape[0] == inserted :
        print(str(index[idx]) + " SUCCESS!!")
        print()
    else:
        print(str(index[idx]) + " ERROR!!")
        print(add_elastic.shape[0],inserted)
        print()
    
    
    
    



# tmp = data_stop.copy()
# add_elastic = preprocess_connect(tmp)

Process in y2005
y2005 SUCCESS!!

Process in y2006
y2006 SUCCESS!!

Process in y2007
y2007 SUCCESS!!

Process in y2008
y2008 SUCCESS!!

Process in y2009
y2009 SUCCESS!!

Process in y2010
y2010 SUCCESS!!

Process in y2011
y2011 SUCCESS!!

Process in y2012
y2012 SUCCESS!!

Process in y2013
y2013 SUCCESS!!

Process in y2014
y2014 SUCCESS!!

Process in y2015
y2015 SUCCESS!!

Process in y2016
y2016 SUCCESS!!

Process in y2017
y2017 SUCCESS!!

Process in y2018
y2018 SUCCESS!!

Process in y2019
y2019 SUCCESS!!

Process in y2005_stop
y2005_stop SUCCESS!!

Process in y2006_stop
y2006_stop SUCCESS!!

Process in y2007_stop
y2007_stop SUCCESS!!

Process in y2008_stop
y2008_stop SUCCESS!!

Process in y2009_stop
y2009_stop SUCCESS!!

Process in y2010_stop
y2010_stop SUCCESS!!

Process in y2011_stop
y2011_stop SUCCESS!!

Process in y2012_stop
y2012_stop SUCCESS!!

Process in y2013_stop
y2013_stop SUCCESS!!

Process in y2014_stop
y2014_stop SUCCESS!!

Process in y2015_stop
y2015_stop SUCCESS!!

Proces

In [ ]:
tttt = y2008_stop.copy()
tttt = tttt[(tttt['Travel Month'].map(type)  != str) &  (tttt['Travel Month'].notnull()) ]
tttt



In [ ]:
y2008_stop.columns

In [ ]:
# y2008_stop
# y2008_stop['Travel Month'].dt.strftime('%Y-%m-%d')
y2008_stop[~(y2008_stop['Travel Month'].isnull())].loc[131312]['Travel Month']

In [ ]:
for j in list(y2008_stop['Travel Month'].unique()):
    print(type(j))

In [ ]:
# y2008_stop.select_dtypes('datetime')
y2008_stop[(y2008_stop['Travel Month'].map(type)  != str) &  (y2008_stop['Travel Month'].notnull()) ]

In [ ]:
ttt = preprocess_connect(y2019_stop.copy())
ttt

In [ ]:
json_str = ttt.to_json(orient='records')
json_records = json.loads(json_str)

In [ ]:
for i,j in enumerate(json_records):
    if(i == 20):
        break;
    print(j)

In [ ]:
# add_data = gendata(json_records,'test')

In [ ]:
# try:
#     helpers.bulk(es, gendata(json_records,'test'))
#     print('ok')
# except Exception as e:
#     print('on')
    

In [ ]:
inserted = 0


for success, info in streaming_bulk(
    client=es, actions=gendata(json_records,'test'),request_timeout=50
    ):
    inserted += success

In [ ]:
inserted

In [ ]:
if __name__ == '__main__':

for name in index:    
        try:
            # create index and index must be lowercase alphabet
            create_index(name.lower())
            print(index,"created successed")
        except Exception as e:
            print(e)
            print(index,"create not success")

In [9]:
results_gen = scan(
    es,
    query=query_body,
    index='y2019',
)

results = list(results_gen)

NameError: name 'scan' is not defined

In [8]:
    results_gen = scan(
        es,
        query=query_body,
        index='y2019',
    )

NameError: name 'scan' is not defined

,Travel Month,Orig,Stop1,Stop2,Stop3,Dest,passenger,Region_orig,country_orig,Region_dest,country_dest
0,2019-03-01,FNT,ATL,ICN,None,BKK,1,North America,United States,Asia,Thailand
1,2019-03-01,LEX,ATL,ICN,None,BKK,1,North America,United States,Asia,Thailand
2,2019-03-01,LEX,DTW,ICN,None,BKK,1,North America,United States,Asia,Thailand
3,2019-03-01,MYR,ATL,ICN,None,BKK,1,North America,United States,Asia,Thailand
4,2019-03-01,DAB,ATL,ICN,None,BKK,4,North America,United States,Asia,Thailand
5,2019-03-01,GSP,DTW,ICN,None,BKK,1,North America,United States,Asia,Thailand
6,2019-03-01,MOB,ATL,ICN,None,BKK,3,North America,United States,Asia,Thailand
7,2019-03-01,GNV,ATL,ICN,None,BKK,4,North America,United States,Asia,Thailand
8,2019-03-01,AGS,ATL,ICN,None,BKK,1,North America,United States,Asia,Thailand
9,2019-03-01,DHN,ATL,ICN,None,BKK,1,North America,United States,Asia,Thailand


In [ ]:
# json.dumps([i['_source'] for i in results ])

In [ ]:
res = [i['_source'] for i in results ]

# es.search(index="test", body=query_body)['hits']['hits']

In [ ]:
res

In [ ]:
dd = pd.DataFrame(res)
dd

In [ ]:
dd[~dd.Stop2.isna()]

In [ ]:
tmp[tmp.Stop1 == 'BKK'].shape

In [ ]:
results

In [ ]:

tttt = preprocess_connect(y2008_stop)
tttt

In [ ]:
y2017_stop

In [ ]:
data = pd.read_excel("./y2017_foreign.xlsx")
country = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/1.-Airport_Lookup_Country-Region.xlsx")
thai = pd.read_excel("../2020 Thesis - Thailand Schdeule 2005-2019/Thailand_Airport_Lookup.xlsx")